In [ ]:
!nvidia-smi

Tue Nov 22 07:15:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, RMSprop, SGD

from sklearn.model_selection import train_test_split

##LOAD DATASET

In [2]:
(trainX, trainY), (testX, testY) = fashion_mnist.load_data()

x = np.concatenate((trainX, testX))
y = np.concatenate((trainY, testY))

np.random.seed(2019)
train_size = 0.8
index = np.random.rand(len(x)) < train_size  # boolean index
trainX, testX = x[index], x[~index]  # index and it's negation
trainY, testY = y[index], y[~index]


print('Train: X = ', trainX.shape)
print('Test: X = ', testX.shape)

4422102/4422102 [==============================] - 0s 0us/step
Train: X =  (55955, 28, 28)
Test: X =  (14045, 28, 28)


##Pre-Proccess on Data


In [3]:
trainX = trainX/255
testX = testX/255

trainX = trainX.reshape(trainX.shape[0],28,28,1)
testX = testX.reshape(testX.shape[0],28,28,1)

In [4]:
print('Train: X = ', trainX.shape)
print('Test: X = ', testX.shape)

Train: X =  (55955, 28, 28, 1)
Test: X =  (14045, 28, 28, 1)


In [5]:
trainY = tf.keras.utils.to_categorical(trainY,10)
testY = tf.keras.utils.to_categorical(testY,10)

print('Train: X = ', trainY[0])
print('Test: X = ', testY.shape)

Train: X =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Test: X =  (14045, 10)


##CREATE THE MODEL FOR FOURTH Architecture 

In [15]:
def model_4th_arch(input_shape = (28,28,1), classes = 10):

  X_input = Input(input_shape)
  X = Conv2D(64, (2,2) , padding="same", activation = 'relu')(X_input)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  X = Dropout(0.25)(X)

  X = Conv2D(64, (2,2),  padding="same", activation = 'relu')(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  X = Dropout(0.25)(X)

  X = Conv2D(64, (2,2),  padding="same", activation = 'relu')(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  X = Dropout(0.25)(X)

  X = Conv2D(64, (2,2),  padding="same", activation = 'relu')(X)
  X = Dropout(0.25)(X)
  X = Flatten()(X)
  X = Dense(64 , activation="relu")(X)
  X = Dropout(0.25)(X)
  X = Dense(10 , activation = 'softmax')(X)

##
  model = models.Model(inputs = X_input, outputs = X)
  return model
  

import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [7]:
model = model_4th_arch()
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        16448     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                         

###Train dataset with SGD optimizer

In [ ]:
model.compile(optimizer = SGD(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(trainX, trainY, epochs = 50, batch_size= 128)

Epoch 1/50
469/469 [==============================] - 12s 8ms/step - loss: 2.3038 - accuracy: 0.1020
Epoch 2/50
469/469 [==============================] - 4s 8ms/step - loss: 2.2952 - accuracy: 0.1171
Epoch 3/50
469/469 [==============================] - 4s 9ms/step - loss: 2.2876 - accuracy: 0.1276
Epoch 4/50
469/469 [==============================] - 4s 8ms/step - loss: 2.2781 - accuracy: 0.1376
Epoch 5/50
469/469 [==============================] - 4s 8ms/step - loss: 2.2664 - accuracy: 0.1495
Epoch 6/50
469/469 [==============================] - 4s 8ms/step - loss: 2.2490 - accuracy: 0.1602
Epoch 7/50
469/469 [==============================] - 4s 8ms/step - loss: 2.2230 - accuracy: 0.1812
Epoch 8/50
469/469 [==============================] - 4s 8ms/step - loss: 2.1854 - accuracy: 0.2020
Epoch 9/50
469/469 [==============================] - 4s 8ms/step - loss: 2.1241 - accuracy: 0.2288
Epoch 10/50
469/469 [==============================] - 4s 8ms/step - loss: 2.0327 - accuracy: 0.260

###Train dataset with Adam optimizer

In [17]:
model.compile(optimizer = Adam(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy','Precision',f1_metric ])
model.fit(trainX, trainY, epochs = 50, batch_size= 128)

Epoch 1/50
438/438 [==============================] - 5s 9ms/step - loss: 0.2323 - accuracy: 0.9149 - precision: 0.9294 - f1_metric: 0.9154
Epoch 2/50
438/438 [==============================] - 4s 9ms/step - loss: 0.2287 - accuracy: 0.9163 - precision: 0.9302 - f1_metric: 0.9164
Epoch 3/50
438/438 [==============================] - 4s 9ms/step - loss: 0.2241 - accuracy: 0.9185 - precision: 0.9318 - f1_metric: 0.9183
Epoch 4/50
438/438 [==============================] - 4s 9ms/step - loss: 0.2235 - accuracy: 0.9167 - precision: 0.9308 - f1_metric: 0.9173
Epoch 5/50
438/438 [==============================] - 4s 9ms/step - loss: 0.2194 - accuracy: 0.9184 - precision: 0.9310 - f1_metric: 0.9185
Epoch 6/50
438/438 [==============================] - 4s 9ms/step - loss: 0.2162 - accuracy: 0.9203 - precision: 0.9328 - f1_metric: 0.9197
Epoch 7/50
438/438 [==============================] - 4s 9ms/step - loss: 0.2158 - accuracy: 0.9185 - precision: 0.9320 - f1_metric: 0.9189
Epoch 8/50
438/438 [

### Test dataset with Adam optimizer

In [18]:
model.compile(optimizer = Adam(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy','Precision',f1_metric ])
model.fit(testX, testY, epochs = 50, batch_size= 128)

Epoch 1/50
110/110 [==============================] - 2s 10ms/step - loss: 0.2570 - accuracy: 0.9080 - precision: 0.9240 - f1_metric: 0.9089
Epoch 2/50
110/110 [==============================] - 1s 9ms/step - loss: 0.2343 - accuracy: 0.9143 - precision: 0.9309 - f1_metric: 0.9158
Epoch 3/50
110/110 [==============================] - 1s 9ms/step - loss: 0.2391 - accuracy: 0.9133 - precision: 0.9284 - f1_metric: 0.9137
Epoch 4/50
110/110 [==============================] - 1s 9ms/step - loss: 0.2353 - accuracy: 0.9154 - precision: 0.9310 - f1_metric: 0.9152
Epoch 5/50
110/110 [==============================] - 1s 9ms/step - loss: 0.2326 - accuracy: 0.9175 - precision: 0.9332 - f1_metric: 0.9182
Epoch 6/50
110/110 [==============================] - 1s 9ms/step - loss: 0.2166 - accuracy: 0.9210 - precision: 0.9355 - f1_metric: 0.9210
Epoch 7/50
110/110 [==============================] - 1s 9ms/step - loss: 0.2210 - accuracy: 0.9200 - precision: 0.9334 - f1_metric: 0.9198
Epoch 8/50
110/110 

## CREATE THE MODEL FOR THE THIRD ARCITECTURE


In [12]:
def model_3th_arch(input_shape = (28,28,1), classes = 10):

  X_input = Input(input_shape)
  X = Conv2D(64, (2,2) , padding="same", activation = 'relu')(X_input)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  X = Dropout(0.25)(X)

  X = Conv2D(64, (2,2),  padding="same", activation = 'relu')(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  X = Dropout(0.25)(X)

  X = Conv2D(64, (2,2),  padding="same", activation = 'relu')(X)
  X = Dropout(0.25)(X)
  X = Flatten()(X)
  X = Dense(64 , activation="softmax")(X)
  X = Dropout(0.25)(X)
  X = Dense(10 , activation = 'softmax')(X)

##
  model = models.Model(inputs = X_input, outputs = X)
  return model

In [13]:
model2 = model_3th_arch()
print(model2.summary())


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 64)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 64)        16448     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                       

###Train dataset with SGD optimizer

In [ ]:
model2.compile(optimizer = SGD(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model2.fit(trainX, trainY, epochs = 50, batch_size= 128)

Epoch 1/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3027 - accuracy: 0.0987
Epoch 2/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3027 - accuracy: 0.0998
Epoch 3/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3026 - accuracy: 0.1006
Epoch 4/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3025 - accuracy: 0.1012
Epoch 5/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3026 - accuracy: 0.0995
Epoch 6/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3024 - accuracy: 0.1011
Epoch 7/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3024 - accuracy: 0.1016
Epoch 8/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3023 - accuracy: 0.1020
Epoch 9/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3022 - accuracy: 0.1046
Epoch 10/50
469/469 [==============================] - 4s 8ms/step - loss: 2.3022 - accuracy: 0.1052

###Train dataset with Adam optimizer

In [19]:
model2.compile(optimizer = Adam(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy','Precision',f1_metric ])
model2.fit(trainX, trainY, epochs = 50, batch_size= 128)

Epoch 1/50
438/438 [==============================] - 5s 9ms/step - loss: 0.4544 - accuracy: 0.8391 - precision: 0.9157 - f1_metric: 0.8406
Epoch 2/50
438/438 [==============================] - 4s 9ms/step - loss: 0.4505 - accuracy: 0.8395 - precision: 0.9160 - f1_metric: 0.8411
Epoch 3/50
438/438 [==============================] - 4s 9ms/step - loss: 0.4412 - accuracy: 0.8451 - precision: 0.9169 - f1_metric: 0.8457
Epoch 4/50
438/438 [==============================] - 4s 9ms/step - loss: 0.4428 - accuracy: 0.8448 - precision: 0.9171 - f1_metric: 0.8442
Epoch 5/50
438/438 [==============================] - 4s 9ms/step - loss: 0.4319 - accuracy: 0.8540 - precision: 0.9207 - f1_metric: 0.8488
Epoch 6/50
438/438 [==============================] - 4s 9ms/step - loss: 0.4237 - accuracy: 0.8553 - precision: 0.9186 - f1_metric: 0.8514
Epoch 7/50
438/438 [==============================] - 4s 9ms/step - loss: 0.4167 - accuracy: 0.8580 - precision: 0.9198 - f1_metric: 0.8557
Epoch 8/50
438/438 [

##Test Dataset with Adam optimizer

In [24]:
model2.compile(optimizer = Adam(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy','Precision',f1_metric ])
model2.fit(testX, testY, epochs = 50, batch_size= 128)

Epoch 1/50
110/110 [==============================] - 2s 9ms/step - loss: 0.3704 - accuracy: 0.8765 - precision: 0.9139 - f1_metric: 0.8773
Epoch 2/50
110/110 [==============================] - 1s 9ms/step - loss: 0.3523 - accuracy: 0.8790 - precision: 0.9177 - f1_metric: 0.8795
Epoch 3/50
110/110 [==============================] - 1s 8ms/step - loss: 0.3481 - accuracy: 0.8854 - precision: 0.9199 - f1_metric: 0.8819
Epoch 4/50
110/110 [==============================] - 1s 8ms/step - loss: 0.3330 - accuracy: 0.8856 - precision: 0.9253 - f1_metric: 0.8872
Epoch 5/50
110/110 [==============================] - 1s 9ms/step - loss: 0.3255 - accuracy: 0.8891 - precision: 0.9249 - f1_metric: 0.8880
Epoch 6/50
110/110 [==============================] - 1s 9ms/step - loss: 0.3238 - accuracy: 0.8884 - precision: 0.9248 - f1_metric: 0.8883
Epoch 7/50
110/110 [==============================] - 1s 8ms/step - loss: 0.3143 - accuracy: 0.8928 - precision: 0.9305 - f1_metric: 0.8930
Epoch 8/50
110/110 [

## CREATE THE MODEL FOR THE FIFTH ARCITECTURE


In [20]:
def model_5th_arch(input_shape = (28,28,1), classes = 10):

  X_input = Input(input_shape)
  X = Conv2D(32, (3,3) , padding="same", activation = 'relu')(X_input)
  X = Conv2D(32, (3,3) , padding="same", activation = 'relu')(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  X = Dropout(0.25)(X)

  X = Conv2D(64, (3,3),  padding="same", activation = 'relu')(X)
  X = Conv2D(64, (3,3),  padding="same", activation = 'relu')(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  X = Dropout(0.25)(X)

  X = Flatten()(X)
  X = Dense(512 , activation="softmax")(X)
  X = Dropout(0.5)(X)
  X = Dense(10 , activation = 'softmax')(X)

##
  model = models.Model(inputs = X_input, outputs = X)
  return model

In [21]:
model3 = model_5th_arch()
print(model3.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 14, 14, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 64)        18496     
                                                           

###Train dataset with SGD optimizer

In [ ]:
model3.compile(optimizer = SGD(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model3.fit(trainX, trainY, epochs = 50, batch_size= 128)

Epoch 1/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.1003
Epoch 2/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.1000
Epoch 3/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.1024
Epoch 4/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.1015
Epoch 5/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.1010
Epoch 6/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.0988
Epoch 7/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.1000
Epoch 8/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accuracy: 0.0977
Epoch 9/50
469/469 [==============================] - 5s 11ms/step - loss: 2.3026 - accuracy: 0.0984
Epoch 10/50
469/469 [==============================] - 5s 10ms/step - loss: 2.3026 - accura

###Train dataset with Adam optimizer

In [22]:
model3.compile(optimizer = Adam(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy','Precision',f1_metric ])
model3.fit(trainX, trainY, epochs = 50, batch_size= 128)

Epoch 1/50
438/438 [==============================] - 6s 11ms/step - loss: 2.0958 - accuracy: 0.2369 - precision: 0.0000e+00 - f1_metric: 0.0000e+00
Epoch 2/50
438/438 [==============================] - 5s 11ms/step - loss: 1.9158 - accuracy: 0.2425 - precision: 0.0000e+00 - f1_metric: 0.0000e+00
Epoch 3/50
438/438 [==============================] - 5s 11ms/step - loss: 1.8447 - accuracy: 0.2437 - precision: 0.7489 - f1_metric: 0.0781
Epoch 4/50
438/438 [==============================] - 5s 11ms/step - loss: 1.7525 - accuracy: 0.2953 - precision: 0.9501 - f1_metric: 0.1346
Epoch 5/50
438/438 [==============================] - 5s 11ms/step - loss: 1.6296 - accuracy: 0.3469 - precision: 0.9270 - f1_metric: 0.2460
Epoch 6/50
438/438 [==============================] - 5s 11ms/step - loss: 1.5549 - accuracy: 0.3636 - precision: 0.9046 - f1_metric: 0.3192
Epoch 7/50
438/438 [==============================] - 5s 11ms/step - loss: 1.5059 - accuracy: 0.3909 - precision: 0.9041 - f1_metric: 0.37

##Test Dataset with Adam optimizer

In [23]:
model3.compile(optimizer = Adam(learning_rate=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy','Precision',f1_metric ])
model3.fit(testX, testY, epochs = 50, batch_size= 128)

Epoch 1/50
110/110 [==============================] - 2s 13ms/step - loss: 0.4773 - accuracy: 0.8455 - precision: 0.9162 - f1_metric: 0.8470
Epoch 2/50
110/110 [==============================] - 1s 11ms/step - loss: 0.4491 - accuracy: 0.8552 - precision: 0.9229 - f1_metric: 0.8563
Epoch 3/50
110/110 [==============================] - 1s 11ms/step - loss: 0.4353 - accuracy: 0.8591 - precision: 0.9261 - f1_metric: 0.8606
Epoch 4/50
110/110 [==============================] - 1s 11ms/step - loss: 0.4484 - accuracy: 0.8549 - precision: 0.9271 - f1_metric: 0.8565
Epoch 5/50
110/110 [==============================] - 1s 11ms/step - loss: 0.4295 - accuracy: 0.8619 - precision: 0.9309 - f1_metric: 0.8627
Epoch 6/50
110/110 [==============================] - 1s 11ms/step - loss: 0.4166 - accuracy: 0.8655 - precision: 0.9368 - f1_metric: 0.8664
Epoch 7/50
110/110 [==============================] - 1s 11ms/step - loss: 0.4219 - accuracy: 0.8632 - precision: 0.9337 - f1_metric: 0.8645
Epoch 8/50
11